In [2]:
from keras.preprocessing.image import load_img, img_to_array
import keras
import tensorflow as tf
import numpy as np
import cv2
import os
import math

In [3]:
files = os.listdir(r'.\AnimeFaceDetecter\Datas\result\face')
keras.backend.set_image_data_format('channels_last')
#keras.backend.set_image_data_format('channels_first')
images = []
for i in files:
    image = load_img(r'.\AnimeFaceDetecter\Datas\result\face\{}'.format(i),False,target_size=(128,128))
    image = img_to_array(image)
    image = image / 256
    images.append(image)
print(image.shape)

(128, 128, 3)


In [4]:
def sample_z(m, n):
    return np.random.uniform(-2., 2., size=[m, n])

In [5]:
X = tf.placeholder(tf.float32, [None, 64,64,3])
Y = tf.placeholder(tf.float32, [None, 1])
Z = tf.placeholder(tf.float32, [None, 100])

In [15]:
def generate(Z):
    bat = 16
    w1 = tf.Variable(tf.random_normal([100, 16384]))
    h1 = tf.matmul(Z,w1)
    re = tf.reshape(h1,[-1,4,4,1024])
    re = tf.layers.batch_normalization(re)
    
    filter = tf.random_normal([4, 4, 512, 1024])
    filter = tf.Variable(filter)
    a = tf.nn.conv2d_transpose(re, filter, [bat,8,8,512], [1,2,2,1], name='a')
    conv_h1 = tf.nn.relu(a)
    
    filter = tf.random_normal([5, 5, 256, 512])
    filter = tf.Variable(filter)
    b = tf.nn.conv2d_transpose(a, filter, [bat,16,16,256], [1,2,2,1], name='b')
    conv_h2 = tf.nn.relu(b)
    
    filter = tf.random_normal([5, 5, 128, 256])
    filter = tf.Variable(filter)
    c = tf.nn.conv2d_transpose(b, filter, [bat,32,32,128], [1,2,2,1],name='c')
    conv_h3 = tf.nn.relu(c)
    
    filter = tf.random_normal([5, 5, 64, 128])
    filter = tf.Variable(filter)
    d = tf.nn.conv2d_transpose(c, filter, [bat,64,64,64], [1,2,2,1],name='d')
    conv_h4 = tf.nn.relu(d)
    
    filter = tf.random_normal([5, 5, 3, 64])
    filter = tf.Variable(filter)
    e = tf.nn.conv2d_transpose(d, filter, [bat,128,128,3], [1,2,2,1],name='e')
    return e

In [16]:
W1 = tf.Variable(tf.random_normal([3,3,3,32],stddev=0.02))
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.02))
W3 = tf.Variable(tf.random_normal([3,3,64,256],stddev=0.02))
W4 = tf.Variable(tf.random_normal([3,3,256,1024],stddev=0.02))
d_list = [W1, W2, W3, W4]
def discriminaster(inputs):
    H1 = tf.nn.conv2d(inputs,W1, strides=[1,4,4,1], padding="SAME")
    H1 = tf.layers.batch_normalization(H1)
    H1 = tf.nn.relu(H1)

    H2 = tf.nn.conv2d(H1,W2, strides=[1,2,2,1], padding="SAME")
    H2 = tf.nn.relu(H2)

    H3 = tf.nn.conv2d(H2,W3, strides=[1,2,2,1], padding="SAME")
    H3 = tf.nn.relu(H3)
    
    H4 = tf.nn.conv2d(H3,W4, strides=[1,2,2,1], padding="SAME")
    H4 = tf.nn.relu(H4)
    
    out = tf.reshape(H4,[-1,16384])
    out = tf.layers.dense(out,256)
    out = tf.layers.dense(out,1)
    return out

In [17]:
G = generate(Z)
D_real = discriminaster(X)
D_gene = discriminaster(G)

In [18]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))
loss_D = loss_D_gene + loss_D_real
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [19]:
train_D = tf.train.AdamOptimizer(0.001).minimize(loss_D)
train_G = tf.train.AdamOptimizer(0.001).minimize(loss_G)

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [22]:
noise = sample_z(16,100)
_, varsD=sess.run([train_D, loss_D], 
              feed_dict={
                  X:np.array(images[:16]),
                  Z: noise})

In [23]:
for i in range(100):
    noise = sample_z(16,100)
    _, varsD=sess.run([train_D, loss_D], 
                  feed_dict={
                      X:np.array(images[:16]),
                      Z: noise})
    _, varsG=sess.run([train_G, loss_G], 
                  feed_dict={
                      X:np.array(images[:16]),
                      Z: noise})
    print(str(i)+"   D : "+str(varsD)+"\t G : "+str(varsG))
    if i % 20 == 0:
        made_image = sess.run(G, feed_dict={Z:sample_z(16,100)})
        c = 0
        for j in made_image:
            c += 1
            cv2.imwrite('{i}-{c}.jpg'.format(i=i,c=c),j)

0   D : 7.6861026e-05	 G : 2025587200.0
1   D : 3.7968994e-12	 G : 353745570.0
2   D : 1.938193e-24	 G : 61185100.0
3   D : 15847850.0	 G : 16656753.0
4   D : 7447305.5	 G : 8934346.0
5   D : 3913250.2	 G : 8869679.0
6   D : 1581617.1	 G : 10079595.0
7   D : 0.0	 G : 10159458.0
8   D : 0.0	 G : 8688796.0
9   D : 0.0	 G : 7125729.5
10   D : 125399.805	 G : 6843062.0
11   D : 215309.36	 G : 9766412.0
12   D : 4798.836	 G : 11919709.0
13   D : 0.0	 G : 12692670.0
14   D : 262200.7	 G : 17434672.0
15   D : 240206.86	 G : 24427476.0
16   D : 0.0	 G : 29781570.0
17   D : 23138.15	 G : 35776870.0
18   D : 195279.73	 G : 44983416.0
19   D : 381766.75	 G : 56406664.0
20   D : 62897.633	 G : 65637980.0
21   D : 0.0	 G : 66803384.0
22   D : 3627195.0	 G : 59147250.0
23   D : 11512973.0	 G : 50347388.0
24   D : 13636076.0	 G : 45403160.0


KeyboardInterrupt: 

In [24]:
made_image = sess.run(G, feed_dict={Z:sample_z(16,100)})

In [25]:
cv2.imwrite('1.jpg',made_image[3])

True

In [63]:

value = tf.random_normal([bat, 4, 4, 1024])
filter = tf.random_normal([4, 4, 512, 1024])
a = tf.nn.conv2d_transpose(value, filter, [bat,8,8,512], [1,2,2,1], name='a')
a

<tf.Tensor 'a_2:0' shape=(16, 8, 8, 512) dtype=float32>

In [64]:
bat = 16
#value = tf.random_normal([bat, 4, 4, 512])
filter = tf.random_normal([5, 5, 256, 512])
b = tf.nn.conv2d_transpose(a, filter, [bat,16,16,256], [1,2,2,1], name='b')
b

<tf.Tensor 'b_1:0' shape=(16, 16, 16, 256) dtype=float32>

In [65]:
bat = 16
value = tf.random_normal([bat, 8, 8, 256])
filter = tf.random_normal([5, 5, 128, 256])
c = tf.nn.conv2d_transpose(b, filter, [bat,32,32,128], [1,2,2,1],name='c')
c

<tf.Tensor 'c_1:0' shape=(16, 32, 32, 128) dtype=float32>

In [74]:
bat = 16
value = tf.random_normal([bat, 16, 16, 128])
filter = tf.random_normal([5, 5, 64, 128])
d = tf.nn.conv2d_transpose(c, filter, [bat,64,64,64], [1,2,2,1],name='d')
d

<tf.Tensor 'd_6:0' shape=(16, 64, 64, 64) dtype=float32>

In [81]:
bat = 16
value = tf.random_normal([bat, 16, 16, 128])
filter = tf.random_normal([5, 5, 3, 64])
e = tf.nn.conv2d_transpose(d, filter, [bat,128,128,3], [1,2,2,1],name='e')
e

<tf.Tensor 'e_11:0' shape=(16, 128, 128, 3) dtype=float32>

In [123]:
sess.run(discriminaster(X), feed_dict={X:i})

FailedPreconditionError: Attempting to use uninitialized value w1
	 [[Node: w1/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](w1)]]

Caused by op 'w1/read', defined at:
  File "C:\Users\haho6\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\haho6\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\haho6\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\haho6\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\haho6\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\haho6\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\haho6\Anaconda3\lib\asyncio\base_events.py", line 422, in ru

In [122]:
def discriminaster(inputs):
    W1 = tf.Variable(tf.random_normal([3,3,3,32],stddev=0.02), name='w1')
    H1 = tf.nn.conv2d(inputs,W1, strides=[1,2,2,1], padding="SAME")
    H1 = tf.layers.batch_normalization(H1)
    H1 = tf.nn.relu(H1)

    W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.02), name='w2')
    H2 = tf.nn.conv2d(H1,W2, strides=[1,2,2,1], padding="SAME")
    H2 = tf.nn.relu(H2)

    W3 = tf.Variable(tf.random_normal([3,3,64,256],stddev=0.02),name='w3')
    H3 = tf.nn.conv2d(H2,W3, strides=[1,2,2,1], padding="SAME")
    H3 = tf.nn.relu(H3)
    
    W4 = tf.Variable(tf.random_normal([3,3,256,1024],stddev=0.02),name='w4')
    H4 = tf.nn.conv2d(H3,W4, strides=[1,2,2,1], padding="SAME")
    H4 = tf.nn.relu(H4)
    
    out = tf.reshape(H4,[-1,16384])
    out = tf.layers.dense(out,256)
    out = tf.layers.dense(out,1)
    return out

In [117]:
i = np.arange(128*128*3*3).reshape(3,128,128,3)